In [1]:
from __future__ import unicode_literals
from textacy import fileio, preprocess, extract, Corpus

In [2]:
summary_docs = fileio.read_file_lines('/home/SMRT-labeled/social-docs')
summary_meta = fileio.read_json_lines('/home/SMRT-labeled/social-meta')

In [3]:
text = [t for t in summary_docs]
meta = [m for m in summary_meta][0]

print(len(text), len(meta))

(1937, 1937)


In [4]:
for k in meta[0]:
    print k, meta[0][k]

Category smrt bus


In [5]:
#load lable data
label_key = u'Category'

label_list = map(lambda x:x[label_key], meta)

In [6]:
import collections

collections.Counter(label_list)

Counter({u'#ref!': 28,
         u'(corporate)': 1,
         u'competitor (bus)': 93,
         u'competitor (buses)': 1,
         u'competitor (taxi)': 33,
         u'competitor (taxis)': 1,
         u'competitor (train)': 61,
         u'competitor (trains)': 30,
         u'competitor bus': 7,
         u'competitor train)': 1,
         u'general mentions': 2,
         u'industry': 390,
         u'smrt': 1,
         u'smrt (corporate)': 88,
         u'smrt bus': 502,
         u'smrt buses': 5,
         u'smrt corporate': 1,
         u'smrt taxi': 68,
         u'smrt taxis': 4,
         u'smrt train': 280,
         u'smrt trains': 339,
         u'taxis': 1})

In [7]:
#re-label: step 1 -- remove 'forum' and text including 'Same as'
text = [x[0] for x in zip(text, label_list) if (x[1] != u'#ref!')]
label_list = [x for x in label_list if ((x != u'#ref!'))]

def hasContent(wds):
    if((wds != u'#ref!') and (len(wds) > 5)):
        return True
    else:
        return False

maskKeyword = map(hasContent, text)
text = [x[1] for x in zip(maskKeyword, text) if(x[0])]
label_list = [x[1] for x in zip(maskKeyword, label_list) if(x[0])]

print(len(text), len(label_list))

(1909, 1909)


In [8]:
collections.Counter(label_list)

Counter({u'(corporate)': 1,
         u'competitor (bus)': 93,
         u'competitor (buses)': 1,
         u'competitor (taxi)': 33,
         u'competitor (taxis)': 1,
         u'competitor (train)': 61,
         u'competitor (trains)': 30,
         u'competitor bus': 7,
         u'competitor train)': 1,
         u'general mentions': 2,
         u'industry': 390,
         u'smrt': 1,
         u'smrt (corporate)': 88,
         u'smrt bus': 502,
         u'smrt buses': 5,
         u'smrt corporate': 1,
         u'smrt taxi': 68,
         u'smrt taxis': 4,
         u'smrt train': 280,
         u'smrt trains': 339,
         u'taxis': 1})

In [9]:
def reLabel(x):
    if(x in [u'(corporate)', u'smrt (corporate)', u'smrt corporate', u'smrt', u'general mentions']):
        return u'SMRT Corporate'
    elif(x in [u'smrt bus', u'smrt buses']):
        return u'SMRT Buses'
    elif(x in [u'smrt train', u'smrt trains']):
        return u'SMRT Trains'
    elif(x in [u'smrt taxi', u'smrt taxis']):
        return u'SMRT Taxis'
    elif(x in [u'competitor (bus)', u'competitor (buses)', u'competitor bus']):
        return u'Buses'
    elif(x in [u'competitor (taxi)', u'competitor (taxis)', 'taxis']):
        return u'Taxis'
    elif(x in [u'competitor (train)', u'competitor (trains)', u'competitor train)']):
        return u'Trains'
    elif(x in [u'industry']):
        return u'Public Transport Industry'
    else:
        return u'Unknown'

label_list = map(reLabel, label_list)
collections.Counter(label_list)

Counter({u'Buses': 101,
         u'Public Transport Industry': 390,
         u'SMRT Buses': 507,
         u'SMRT Corporate': 93,
         u'SMRT Taxis': 72,
         u'SMRT Trains': 619,
         u'Taxis': 35,
         u'Trains': 92})

In [10]:
unique_label = list(set(label_list))
label2id = dict(zip(unique_label, range(len(unique_label))))
label_id = map(lambda x:label2id[x], label_list)

corpus_train = Corpus('en', texts = text)

In [13]:
#topic model
from textacy import vsm

terms_lists = (doc.to_terms_list(ngrams={2, 3}, named_entities=True, as_strings=True) for doc in corpus_train)
doc_term_matrix, id2term = vsm.doc_term_matrix(terms_lists, weighting='tf', normalize=True, smooth_idf=True, min_df=3, max_df=0.95, max_n_terms=15000)

In [14]:
doc_term_matrix

<1909x4527 sparse matrix of type '<type 'numpy.float64'>'
	with 36899 stored elements in Compressed Sparse Row format>

In [15]:
#MLPClassifier requires sklearn >=0.18
#classifier
#from sklearn.neural_network import MLPClassifier
#regression
from sklearn.neural_network import MLPRegressor
#use sknn instead
#from sknn.mlp import Classifier, Layer
import numpy as np

def id2array(id, a_len):
    result = np.zeros(a_len)
    result[id] = 1
    return result

labeltype = len(label2id.keys())

msg_x = doc_term_matrix.toarray()
msg_y = np.array([id2array(x, labeltype) for x in label_id])

In [16]:
clf = MLPRegressor(activation='logistic', solver='adam', alpha=1e-5, batch_size=20, hidden_layer_sizes=(500, 120), random_state=1)
clf.fit(msg_x, msg_y)

MLPRegressor(activation=u'logistic', alpha=1e-05, batch_size=20, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(500, 120), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver=u'adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [17]:
clf.score(msg_x, msg_y)

0.97855049689833751

In [18]:
#save model
from sklearn.externals import joblib

savefiles = joblib.dump(clf, '/home/pretrain-model/category-ann-reg-social.pkl') 

#save meta
fileio.write_json(label2id, '/home/pretrain-model/category-label2id-ann-reg-social')
fileio.write_json(id2term, '/home/pretrain-model/category-id2term-ann-reg-social')